In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model


2022-09-28 09:46:30.419220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-28 09:46:30.419249: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# load data

In [3]:
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
x_train[0].shape

(28, 28)

In [5]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
print(x_train[0].shape)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

(28, 28, 1)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

# model

In [8]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         


2022-09-28 09:46:33.865764: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-28 09:46:33.865879: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-28 09:46:33.865957: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (amaurylc-ThinkPad-T460): /proc/driver/nvidia/version does not exist
2022-09-28 09:46:33.866530: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


                                                                 
 dropout (Dropout)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 10)                16010     
                                                                 
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
_________________________________________________________________


In [9]:
batch_size = 128
epochs = 30

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/30
422/422 [==============================] - 29s 66ms/step - loss: 0.3665 - accuracy: 0.8900 - val_loss: 0.0948 - val_accuracy: 0.9743
Epoch 2/30
422/422 [==============================] - 27s 63ms/step - loss: 0.1122 - accuracy: 0.9656 - val_loss: 0.0587 - val_accuracy: 0.9847
Epoch 3/30
422/422 [==============================] - 27s 64ms/step - loss: 0.0842 - accuracy: 0.9741 - val_loss: 0.0489 - val_accuracy: 0.9872
Epoch 4/30
422/422 [==============================] - 27s 65ms/step - loss: 0.0715 - accuracy: 0.9786 - val_loss: 0.0421 - val_accuracy: 0.9893
Epoch 5/30
422/422 [==============================] - 27s 65ms/step - loss: 0.0624 - accuracy: 0.9814 - val_loss: 0.0406 - val_accuracy: 0.9880
Epoch 6/30
422/422 [==============================] - 28s 65ms/step - loss: 0.0583 - accuracy: 0.9822 - val_loss: 0.0373 - val_accuracy: 0.9897
Epoch 7/30
422/422 [==============================] - 28s 66ms/step - loss: 0.0529 - accuracy: 0.9833 - val_loss: 0.0364 - val_accuracy:

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.0210556760430336
Test accuracy: 0.9926999807357788


In [11]:
model.save('chiffre.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model
# returns a compiled model
# identical to the previous one
#model = load_model('my_model.h5')